In [128]:
import os
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
print(PROJ_ROOT)

/home/bruno/Desktop/coarse-discourse-validation


In [129]:
import pandas as pd
import nltk
import numpy as np

df_path = os.path.join(PROJ_ROOT, 'data', 'interim', 'clean_data.pkl')

df = pd.read_pickle(df_path)

df.head()

,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...


In [130]:
df_sample = df.iloc[0:1000]

df_sample.head()

,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...


# Features Exploration

In here, we'll be working on the features described on the paper.

## Structure

> We calculate several features related to the structure of the comment and its position. One feature is the depth of the comment according to Reddit’s threaded structure, which we collect as both a raw count and normalized by the number of comments in the discussion. We also calculate number of sentences, number of words, and number of characters of both the body and the title of the comment. We computed these values for both the current comment and the parent comment.

### Post depth

In [131]:
df_sample['post_depth'].value_counts()

1.0     412
2.0     210
3.0     129
0.0     107
4.0      73
5.0      31
6.0      15
7.0       8
8.0       6
10.0      5
9.0       4
Name: post_depth, dtype: int64

In [132]:
# df_sample['post_depth_normalized'] = df_sample['post_depth'] / df_sample['comments_in_discussion']

### Number of sentences

We use NLTK.tokenize to get setences tokenized and then counted.

In [133]:
df_sample['body'].iloc[42]

"You won't regret keeping it. As far as longevity goes, it is almost indefinite. You will wear out parts like springs, mags, and maybe even the grips. The slide, frame, and barrel will likely outlive you. Colt makes great guns. How many companies who produced pistols 100 years ago still have their products found on gun shop shelves in working and safe condition?"

In [134]:
nltk.download('punkt')

nltk.tokenize.sent_tokenize(df_sample['body'].iloc[42])

[nltk_data] Downloading package punkt to /home/bruno/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


["You won't regret keeping it.",
 'As far as longevity goes, it is almost indefinite.',
 'You will wear out parts like springs, mags, and maybe even the grips.',
 'The slide, frame, and barrel will likely outlive you.',
 'Colt makes great guns.',
 'How many companies who produced pistols 100 years ago still have their products found on gun shop shelves in working and safe condition?']

In [135]:
sent_number = lambda b : len(nltk.tokenize.sent_tokenize(b))

df_sample['n_sentences'] = df_sample['body'].map(sent_number)

df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url,n_sentences
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,4
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5


### Number of words

In [136]:
nltk.tokenize.word_tokenize(df_sample['body'].iloc[42])

['You',
 'wo',
 "n't",
 'regret',
 'keeping',
 'it',
 '.',
 'As',
 'far',
 'as',
 'longevity',
 'goes',
 ',',
 'it',
 'is',
 'almost',
 'indefinite',
 '.',
 'You',
 'will',
 'wear',
 'out',
 'parts',
 'like',
 'springs',
 ',',
 'mags',
 ',',
 'and',
 'maybe',
 'even',
 'the',
 'grips',
 '.',
 'The',
 'slide',
 ',',
 'frame',
 ',',
 'and',
 'barrel',
 'will',
 'likely',
 'outlive',
 'you',
 '.',
 'Colt',
 'makes',
 'great',
 'guns',
 '.',
 'How',
 'many',
 'companies',
 'who',
 'produced',
 'pistols',
 '100',
 'years',
 'ago',
 'still',
 'have',
 'their',
 'products',
 'found',
 'on',
 'gun',
 'shop',
 'shelves',
 'in',
 'working',
 'and',
 'safe',
 'condition',
 '?']

In [137]:
words_number = lambda b : len(nltk.tokenize.word_tokenize(b))

df_sample['n_words'] = df_sample['body'].map(words_number)

df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url,n_sentences,n_words
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29,499
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,22
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,4,82
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,26
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5,115


### Number of characters

Here we create a tokenizer from a RegEx to remove punctuaions and whitespaces, so we can count only characters.

In [138]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

words = tokenizer.tokenize(df_sample['body'].iloc[42])

sum(map(len, words))

289

In [139]:
char_number = lambda b : sum(map(len, tokenizer.tokenize(b)))

df_sample['n_char'] = df_sample['body'].map(char_number)

df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url,n_sentences,n_words,n_char
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29,499,2009
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,22,74
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,4,82,339
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,26,84
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5,115,434


### Parents number of sentences, words and characters

In [140]:
df_sample['parent_n_sentences'] = 0
df_sample['parent_n_words'] = 0
df_sample['parent_n_char'] = 0

for i, row in df_sample.iterrows():
    try:
        parent = df_sample[(df_sample['thread_title'] == row['thread_title'])][df_sample['id'] == row['in_reply_to']].iloc[0]
        df_sample.at[i, 'parent_n_sentences'] = parent['n_sentences']
        df_sample.at[i, 'parent_n_words'] = parent['n_words']
        df_sample.at[i, 'parent_n_char'] = parent['n_char']
    except:
        pass
    
df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url,n_sentences,n_words,n_char,parent_n_sentences,parent_n_words,parent_n_char
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29,499,2009,0,0,0
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,22,74,29,499,2009
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,4,82,339,2,22,74
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,26,84,4,82,339
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5,115,434,0,0,0
